In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 8.6MB/s 


In [2]:
import sys
sys.path.insert(0,'/content/gdrive/My Drive/EECS545')

In [3]:
import os

# TODO: Fill in the Google Drive path where you uploaded the files 
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = './EECS545/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['COVID_combined', 'Normal', 'maskHE2_labels.csv', 'test', 'validation', 'ReadMe.gdoc', 'training.ipynb', 'training', 'train_dataset.npy', 'valid_dataset.npy', 'train_dataset_jiahong.npy', 'valid_dataset_jiahong.npy', 'Model_LungSegmentation', 'Model_Without_Lungseg', 'images_nomask', 'HE_output2.zip', 'Things to be done for Final Project.gdoc', 'Siamese_Learning.ipynb', 'grad_labels.csv', 'Matching_code', 'val_labels.csv', 'test_labels.csv', 'train_labels.csv', 'Copy of Siamese_Learning.ipynb', 'COVID-19 X-Ray Image Classification Using Transfer Learning and Contrastive Learning.gslides', 'Training_VGG16_CrossValidation_WithLungMasking.ipynb', 'grad_labels_4.csv', 'grad_labels_new.csv', 'create_label.ipynb']


In [4]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image

import copy

In [ ]:
training_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])
validation_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_csv = os.path.join(GOOGLE_DRIVE_PATH,'train_labels.csv')
val_csv = os.path.join(GOOGLE_DRIVE_PATH,'val_labels.csv')
test_csv = os.path.join(GOOGLE_DRIVE_PATH,'test_labels.csv')

class_numbers = 2

test_path = os.path.join(GOOGLE_DRIVE_PATH,"test")
train_path = os.path.join(GOOGLE_DRIVE_PATH,"training")
val_path = os.path.join(GOOGLE_DRIVE_PATH,"validation")

batch_size = 32

In [ ]:
# PyTorch modules
import torch
from torch import nn 
from torch.utils import data 
import torch.nn.functional as F 
from torchvision import transforms, models

# other modules
import os
import pandas as pd
import numpy as np
from PIL import Image
import random
import pydicom
import cv2
class CheXpert_Dataset(data.Dataset):
  
    def __init__(self, patient_table, image_dir, epoch_size, transform=None):
 
        self.patient_table = patient_table
        self.image_dir = image_dir # note, images are already 8-bit, full-sized JPEGs
        self.transform = transform
        self.epoch_size = epoch_size 
        if self.transform is None:
            self.transform = transforms.Compose([transforms.ToTensor()])
 
    def __len__(self):
        return self.epoch_size
 
    def __getitem__(self, idx): 
        
        patient_table = self.patient_table
        # separate tables for abnormal lung labels 0 and 1
        table_A = patient_table[patient_table['encoded_labels'] == 0]
        table_B = patient_table[patient_table['encoded_labels'] == 1]

        # goal is 50:50 distribution of change vs no change
        change_binary = random.randint(0,1) 

        # if no change 
        if change_binary == 0:

            # select two random images 
            table_A_or_B = random.randint(0,1)

            if table_A_or_B == 0:
                pick0 = random.choice(range(len(table_A)))
                pick1 = random.choice(range(len(table_A)))
                img0 = Image.open(self.image_dir + '/' + table_A.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_A.iloc[pick1]['Images'])

                abnormal_lung_0 = 0
                abnormal_lung_1 = 0

            if table_A_or_B == 1:
                pick0 = random.choice(range(len(table_B)))
                pick1 = random.choice(range(len(table_B)))
                img0 = Image.open(self.image_dir + '/' + table_B.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

                abnormal_lung_0 = 1
                abnormal_lung_1 = 1

        # if change -- note: direction of change does not matter in this step
        if change_binary == 1: 

            pick0 = random.choice(range(len(table_A)))
            pick1 = random.choice(range(len(table_B)))
            img0 = Image.open(self.image_dir + '/'+ table_A.iloc[pick0]['Images'])
            img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

            abnormal_lung_0 = 0
            abnormal_lung_1 = 1

        # 0 for no change, 1 for change (difference between labels)
        if abnormal_lung_0 == abnormal_lung_1:
            change_label = 0
        else:
            change_label = 1
 
        meta = {"path0": patient_table.iloc[pick0]['Images'],
                "path1": patient_table.iloc[pick1]['Images'],
                "abnormal_lung_0": abnormal_lung_0,
                "abnormal_lung_1": abnormal_lung_1,
                }

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1) 

        return img0, img1, change_label, meta

In [ ]:
class SiameseNetwork_DenseNet121(nn.Module):

    def __init__(self):
        super(SiameseNetwork_DenseNet121, self).__init__()
        self.cnn0 = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True)
        # map last layer from 1000 node outcome to three nodes
        #self.cnn0.fc = nn.Linear(1000, 3) # mapping input image to a 3D space

    def forward_once(self, x):
        output = self.cnn0(x)
        return output

    def forward(self, input0, input1):
        output0 = self.forward_once(input0)
        output1 = self.forward_once(input1)
        return output0, output1
  

 

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    Modified from: https://hackernoon.com/facial-similarity-with-siamese-networks-in-pytorch-9642aa9db2f7
    """ 

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive


In [ ]:
class MSELoss(torch.nn.Module):
    """
    MSE loss
    """ 

    def __init__(self):
        super(MSELoss, self).__init__()

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
    
        return F.mse_loss(euclidean_distance,label)

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable
  
# other modules 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import statistics
import pickle
from tqdm import tqdm
working_path = '/content/gdrive/My Drive/EECS545/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
training_table = pd.read_csv(train_csv)
validation_table = pd.read_csv(val_csv)
table_A = validation_table[validation_table['encoded_labels'] == 1]

In [ ]:
img0 = Image.open(test_path + '/'+ table_A.iloc[0]['Images'])

In [ ]:
training_siamese_dataset = CheXpert_Dataset(patient_table = training_table, 
                                        image_dir = train_path, 
                                        epoch_size = 1000,
                                        transform = training_transforms)
 
training_dataloader = torch.utils.data.DataLoader(training_siamese_dataset, 
                                                  batch_size=8, 
                                                  shuffle=True, 
                                                  num_workers=1)

In [ ]:
validation_siamese_dataset = CheXpert_Dataset(patient_table = validation_table, 
                                        image_dir = val_path, 
                                        epoch_size = 1000,
                                        transform = validation_transforms)
 
validation_dataloader = torch.utils.data.DataLoader(validation_siamese_dataset, 
                                                  batch_size=8, 
                                                  shuffle=True, 
                                                  num_workers=1)
import os.path
from os import path

In [ ]:
def siamese_training(training_dataloader, validation_dataloader, working_path, output_folder_name, learning_rate = 0.00002):
    '''
    - Implements siamese network training/validation with return of model
    - Implementation uses early stopping, saving the model with the best validation loss
 
    Arguments
    - training and validation dataloader objects
    - output_folder_name: directory to save model and annotations
    - learning_rate: for Adam optimizer
    ''' 
    net = SiameseNetwork_DenseNet121().cuda()
    criterion = ContrastiveLoss()
    criterion.margin = 50.0  # contrastive loss function margin
    optimizer = optim.Adam(net.parameters(),lr = learning_rate)
 
    # Initialization
    num_epochs = 200
    training_losses, validation_losses = [], []

    # Early stopping initialization
    epochs_no_improve = 0
    max_epochs_stop = 30 # "patience" - number of epochs with no improvement in validation loss after which training stops
    validation_loss_min = np.Inf
    history = []

    output_dir = '/content/drive/MyDrive/EECS545/matching_results'
    #os.mkdir(output_dir)
    if path.exists(output_dir+'/PXS_score_model.pth'):
      net.load_state_dict(torch.load(output_dir+'/PXS_score_model.pth'))
    f = open(output_dir + "/history_training.txt", 'a+')
    f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" + "Training starting now...\n")
    f.close()

    for epoch in range(0, num_epochs):
        print("epoch training started...")

        # keep track of training and validation loss each epoch
        training_loss = 0
        validation_loss = 0

        # keep track of euclidean_distance and label history each epoch
        training_euclidean_distance_history = []
        training_label_history = []
        validation_euclidean_distance_history = []
        validation_label_history = []

        # model set to train
        net.train()
          
        # training loop
        for i, data in tqdm(enumerate(training_dataloader, 0)):
            # train neural network
            img0, img1, label, meta = data
            img0 = np.repeat(img0, 3, 1) # repeat grayscale image in 3 channels (DenseNet requires 3-channel input) 
            img1 = np.repeat(img1, 3, 1)
            img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()  # send tensors to the GPU
            optimizer.zero_grad() # clear gradients
            output0, output1 = net.forward(img0, img1)
            loss_contrastive = criterion(output0, output1, label.float())
            loss_contrastive.backward()
            optimizer.step()

            # keep track of training loss
            training_loss += loss_contrastive.item()

            # save euclidean distance and label history 
            net.eval()
            output0, output1 = net.forward(img0, img1)
            net.train()
            euclidean_distance = F.pairwise_distance(output0, output1)
            euclid_tmp = torch.Tensor.numpy(euclidean_distance.detach().cpu())
            training_euclidean_distance_history.extend(euclid_tmp)

            label_tmp = torch.Tensor.numpy(label.cpu())
            training_label_history.extend(label_tmp)

            print("training loop " + str(i) + " completed")

        else:
            print("validation started...")
            #turn off gradients for validation
            with torch.no_grad(): 
                net.eval() # set evaluation mode

                # determine validation loss
                for j, data2 in tqdm(enumerate(validation_dataloader, 0)):
                    img0, img1, label, meta = data2
                    img0 = np.repeat(img0, 3, 1) # repeat grayscale image in 3 channels (DenseNet requires 3-channel input) 
                    img1 = np.repeat(img1, 3, 1)
                    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
                    output0, output1 = net.forward(img0, img1)
                    loss_contrastive = criterion(output0, output1, label.float())
                    validation_loss += loss_contrastive.item()
                     
                    # save euclidean distance and label history 
                    euclidean_distance = F.pairwise_distance(output0, output1)                
                    euclid_tmp = torch.Tensor.numpy(euclidean_distance.detach().cpu())
                    validation_euclidean_distance_history.extend(euclid_tmp)

                    label_tmp = torch.Tensor.numpy(label.cpu())
                    validation_label_history.extend(label_tmp)

                    print("validation loop " + str(j) + " completed")

            # calculate average training and validation losses (averaged across batches for the epoch)
            training_loss_avg = training_loss/len(training_dataloader)
            validation_loss_avg = validation_loss/len(validation_dataloader)

            # Save the model if validation loss decreases
            if validation_loss_avg < validation_loss_min:
                # save model
                torch.save(net.state_dict(), output_dir + "/PXS_score_model.pth")
                # track improvement
                epochs_no_improve = 0
                validation_loss_min = validation_loss_avg
                best_epoch = epoch
 
            # Otherwise increment count of epochs with no improvement
            else: 
                epochs_no_improve += 1 
                # Trigger EARLY STOPPING
                if epochs_no_improve >= max_epochs_stop:
                    print(f'\nEarly Stopping! Total epochs (starting from 0): {epoch}. Best epoch: {best_epoch} with loss: {validation_loss_min:.2f}')
                    # Load the best state dict (at the early stopping point)
                    net.load_state_dict(torch.load(output_dir + "/PXS_score_model.pth"))
                    # attach the optimizer
                    net.optimizer = optimizer

                    # save history with pickle
                    with open(output_dir + "/history_training.pckl", "wb") as f:
                        pickle.dump(history, f)

                    f = open(output_dir + "/history_training.txt", 'a+')
                    f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" +
                            "Early stopping! Total epochs (starting from 0): {:.0f}\n".format(epoch) +
                            "Best epoch: {:.0f}\n".format(best_epoch) +
                            "Validation loss at best epoch: {:.3f}\n".format(validation_loss_min)
                            )
                    f.close()

                    return net, history, output_dir # break the function

        # after each Epoch

        # append to lists for graphing
        training_losses.append(training_loss_avg)
        validation_losses.append(validation_loss_avg)

        # training euclidean distance stats

        # extract euclidean distances if label is 0 or 1
        euclid_if_0 = [b for a, b in zip(training_label_history, training_euclidean_distance_history) if a == 0]
        euclid_if_1 = [b for a, b in zip(training_label_history, training_euclidean_distance_history) if a == 1]
        euclid_if_0 = np.array(euclid_if_0).tolist()
        euclid_if_1 = np.array(euclid_if_1).tolist()
        
        # summary statistics for euclidean distances
        mean_euclid_0t = statistics.mean(euclid_if_0) 
        std_euclid_0t = statistics.pstdev(euclid_if_0) # population stdev
        mean_euclid_1t = statistics.mean(euclid_if_1)
        std_euclid_1t = statistics.pstdev(euclid_if_1) # population stdev
        euclid_diff_t = mean_euclid_1t - mean_euclid_0t

        # validation euclidean distance stats

        # extract euclidean distances if label is 0 or 1
        euclid_if_0 = [b for a, b in zip(validation_label_history, validation_euclidean_distance_history) if a == 0]
        euclid_if_1 = [b for a, b in zip(validation_label_history, validation_euclidean_distance_history) if a == 1]
        euclid_if_0 = np.array(euclid_if_0).tolist()
        euclid_if_1 = np.array(euclid_if_1).tolist()
        
        # summary statistics for euclidean distances
        mean_euclid_0v = statistics.mean(euclid_if_0) 
        std_euclid_0v = statistics.pstdev(euclid_if_0) # population stdev
        mean_euclid_1v = statistics.mean(euclid_if_1)
        std_euclid_1v = statistics.pstdev(euclid_if_1) # population stdev
        euclid_diff_v = mean_euclid_1v - mean_euclid_0v

        # store in history list
        history = [training_losses, validation_losses, euclid_diff_t, euclid_diff_v]
 
        # save history with pickle
        with open(output_dir + "/history_training.pckl", "wb") as f:
            pickle.dump(history, f)
 
        print("Epoch number: {:.0f}\n".format(epoch),
            "Training loss: {:.3f}\n".format(training_loss_avg),
            "Validation loss: {:.3f}\n".format(validation_loss_avg),
            "\nTraining \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0t),
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0t),
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1t),
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1t),
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_t),
            "\nValidation \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0v),
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0v),
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1v),
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1v),
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_v)
            )

        # write history to file
        f = open(output_dir + "/history_training.txt", 'a+')
        f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" +
            "Epoch number: {:.0f}\n".format(epoch) +
            "Training loss: {:.3f}\n".format(training_loss_avg) +
            "Validation loss: {:.3f}\n".format(validation_loss_avg) +
            "\nTraining \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0t) +
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0t) +
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1t) +
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1t) +
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_t) +
            "\nValidation \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0v) + 
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0v) + 
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1v) + 
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1v) +
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_v) + "\n"
            )
        f.close()
 
    # Load the best state dict (at the early stopping point)
    net.load_state_dict(torch.load(output_dir + "/PXS_score_model.pth"))
    # After training through all epochs attach the optimizer
    net.optimizer = optimizer

    # Return the best model and history
    print(f'\nAll Epochs completed! Total epochs (starting from 0): {epoch}. Best epoch: {best_epoch} with validation loss: {validation_loss_min:.2f}')

    return net, history, output_dir

# siamese training 
net, history, output_dir = siamese_training(training_dataloader = training_dataloader, 
                                            validation_dataloader = validation_dataloader, 
                                            working_path = working_path,
                                            output_folder_name = 'CheXpert_model')
 
# Training/validation learning curves
plt.title("Number of Training Epochs vs. Contrastive Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Contrastive Loss")
plt.plot(range(0, len(history[0])), history[0], label = "Training loss")
plt.plot(range(0, len(history[1])), history[1], label = "Validation loss")
plt.legend(frameon=False)
plt.savefig(output_dir + "/Learning_curve.png")
plt.close()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


epoch training started...


1it [00:56, 56.01s/it]

training loop 0 completed


2it [01:00, 40.44s/it]

...

validation loop 120 completed
validation loop 121 completed


124it [00:22,  5.34it/s]

validation loop 122 completed
validation loop 123 completed


125it [00:23,  5.43it/s]


validation loop 124 completed

Early Stopping! Total epochs (starting from 0): 44. Best epoch: 14 with loss: 162.19


In [ ]:
print(net)

SiameseNetwork_DenseNet121(
  (cnn0): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-

**Test Phase**

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable
  
# other modules 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import statistics
import pickle
from tqdm import tqdm
from PIL import Image
import seaborn as sns
import scipy
from sklearn import metrics
output_dir = '/content/drive/MyDrive/EECS545/matching_results'
net = SiameseNetwork_DenseNet121().cuda()
net.load_state_dict(torch.load(output_dir+'/PXS_score_model.pth'))
def img_processing(input_image):
    '''
    processes PIL image file
    '''
    
    transf = transforms.Compose([
        transforms.Resize(336), # maybe better to factor out resize step
        transforms.CenterCrop(320),
        transforms.ToTensor()
    ])

    output_image= transf(input_image)
    output_image = np.repeat(output_image, 3, 0)
    output_image= output_image[np.newaxis, ...]
    output_image = Variable(output_image).cuda()

    return output_image
def anchor_inference(img_anchor, image_path, net):
    '''
    takes img_anchor list object, image path of interest, and siamese network
    '''

    img_comparison = img_processing(Image.open(image_path))

    save_euclidean_distance = []
    net.eval()
    for j in range(len(img_anchor)):
        output1, output2 = net.forward(img_anchor[j], img_comparison)
        euclidean_distance = F.pairwise_distance(output1, output2)
        save_euclidean_distance.append(euclidean_distance.item())

    # take median euclidean distance compared to the the pool of normals
    return statistics.mean(save_euclidean_distance)

Downloading: "https://github.com/pytorch/vision/archive/v0.5.0.zip" to /root/.cache/torch/hub/v0.5.0.zip
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [ ]:
training_table = pd.read_csv(train_csv)
table_A = training_table[training_table['encoded_labels'] == 1].sample(n=100)
table_B = training_table[training_table['encoded_labels'] == 0].sample(n=100)

In [ ]:
test_table = pd.read_csv(test_csv)

In [ ]:
img_anchor_1 = []
for a in tqdm(range(len(table_A))):
    image_path = train_path+'/' + table_A.iloc[a]['Images']
    img_anchor_1.append(img_processing(Image.open(image_path)))


100%|██████████| 100/100 [00:34<00:00,  2.94it/s]


In [ ]:
test_table

,Images,labels,encoded_labels
0,COVID-1022.png,COVID,0
1,COVID-1351.png,COVID,0
2,COVID-1343.png,COVID,0
3,COVID-1331.png,COVID,0
4,COVID-1272.png,COVID,0
...,...,...,...
718,Normal-990.png,NORMAL,1
719,Normal-995.png,NORMAL,1
720,Normal-978.png,NORMAL,1
721,Normal-976.png,NORMAL,1


In [ ]:
img_anchor_0 = []
for a in tqdm(range(len(table_B))):
    image_path = train_path+'/' + table_B.iloc[a]['Images']
    print(table_B.iloc[a]['Images'])
    img_anchor_0.append(img_processing(Image.open(image_path)))


  0%|          | 0/100 [00:00<?, ?it/s]

COVID-2076.png



  1%|          | 1/100 [00:00<00:33,  3.00it/s]

COVID-1884.png



  2%|▏         | 2/100 [00:00<00:33,  2.89it/s]

...


 98%|█████████▊| 98/100 [00:32<00:00,  3.23it/s]

COVID-1954.png



 99%|█████████▉| 99/100 [00:33<00:00,  3.18it/s]

COVID-2356.png



100%|██████████| 100/100 [00:33<00:00,  2.97it/s]


In [ ]:
correct=0
test_index=[]
for a in tqdm(range(len(test_table))):
   image_path = test_path+'/' + test_table.iloc[a]['Images']
   PXS_0 = anchor_inference(img_anchor_0,image_path, net)
   PXS_1 = anchor_inference(img_anchor_1,image_path, net)
   if PXS_1>PXS_0:
     label=0
   else:
     label=1
   test_index.append(label)
   if label==test_table.iloc[a]['encoded_labels']:
     correct+=1
print(correct/len(test_table))



100%|██████████| 723/723 [1:16:31<00:00,  6.35s/it]

0.9488243430152143


In [ ]:
print(correct/len(test_table))

0.9488243430152143


In [ ]:
from scipy.io import savemat

mdic = {"Result": np.asarray(test_index)}
savemat("Matching_Test_Index.mat", mdic)

In [ ]:
np.asarray(test_index)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(correct/len(test_table))

0.9488243430152143


In [ ]:
PXS_0

1.882044540643692

In [ ]:
if PXS_1>PXS_0:
     label=0
else:
     label=0
if label==test_table.iloc[0]['encoded_labels']:
    correct+=1

In [ ]:
label

1

In [6]:
from scipy.io import loadmat
x = loadmat(GOOGLE_DRIVE_PATH+'/Matching_code/Matching_Test_Index.mat')

In [7]:
results = x["Result"]
test_csv = os.path.join(GOOGLE_DRIVE_PATH,'test_labels.csv')
labels = pd.read_csv(test_csv)['encoded_labels'].values

In [9]:
print(results.size)
print(labels.size)

723
723


In [10]:
import sklearn.metrics as sm
results=results.reshape((723,1))
labels=labels.reshape((723,1))
scores = sm.precision_recall_fscore_support(np.array(labels),np.array(results) , average='binary')
#print(scores)
precision = scores[0]
recall = scores[1]
fscore = scores[2]
print(fscore)
conf_matrix = sm.confusion_matrix(labels, results)
sensitivity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[1,0])
specificity = conf_matrix[1][1]/(conf_matrix[0,1]+conf_matrix[1,1])
print(sensitivity)
print(specificity)

0.9502018842530282
0.9736842105263158
0.926509186351706
